In [1]:
import os
from dotenv import load_dotenv, find_dotenv 
_ = load_dotenv(find_dotenv()) 
openai_api_key = os.environ["OPENAI_API_KEY"]

In [2]:
from langchain_core.prompts import ChatPromptTemplate 
from langchain_openai import ChatOpenAI  
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template("Tell me funny joke about {person}") 

output_parser = StrOutputParser() 

model = ChatOpenAI()

In [3]:
chain = prompt | model | output_parser 

print(chain.invoke({"person": "Michael  Jackson"}))

Why did Michael Jackson go to Walmart?

Because he heard they had great deals on kids' clothes!


In [4]:
# .bind() 

chain = prompt | model.bind(stop = ["Michael Jackson"]) | output_parser

print(chain.invoke({"person": "Michael Jackson"}))

Why did 


#### **Combined Chain**

In [5]:
historian_prompt = ChatPromptTemplate.from_template("What Was {person} ?")

composed_chain = {"person" : chain} | historian_prompt | model | StrOutputParser()

In [6]:
composed_chain.invoke({"person" : "Kanye"})

'This is a play on words, as "on the house" typically means that drinks are free or provided by the establishment. Kanye West bringing a ladder to the bar suggests that he thought the drinks were literally on top of the building or structure.'

In [7]:
from operator import itemgetter 
from langchain_core.prompts import ChatPromptTemplate 
from langchain_core.output_parsers import StrOutputParser 
from langchain_openai import ChatOpenAI 

prompt1 = ChatPromptTemplate.from_template("What is the country {politician} is from ?") 
prompt2 = ChatPromptTemplate.from_template(
    "What continent is the country {country} in ? respond in {language}"
)

chain1 = prompt1 | model | StrOutputParser() 

chain2 = (
    {"country" : chain1, "language" : itemgetter("language")} 
    | prompt2 
    | model
    | StrOutputParser() 
)

chain2.invoke({"politician" : "Miterrand", "language": "French"})

"François Mitterrand était originaire d'Europe, plus précisément du continent européen."

#### **LCEL chain at work in a typical RAG app**

In [8]:
import bs4 
from langchain import hub
from langchain_chroma import Chroma 
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser 
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings 
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader(
    web_path=("https://lilianweng.github.io/posts/2023-06-23-agent/"), 
    bs_kwargs=dict(
        parse_only = bs4.SoupStrainer(
            class_ = ("post-content", "post-title", "post-header")
        ),
    )
)

docs = loader.load() 

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200) 

splits = text_splitter.split_documents(docs) 

vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever() 

prompt = hub.pull("rlm/rag-prompt") 

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
     
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt 
    | model 
    | StrOutputParser()
)

USER_AGENT environment variable not set, consider setting it to identify your requests.
d:\Code\Langchain\venv\Lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [9]:
rag_chain.invoke("What is Decompostion ?")

'Decomposition is the process of breaking down a problem into multiple thought steps or manageable tasks. This can help in generating multiple thoughts per step and creating a tree structure for problem-solving. Different techniques, such as using simple prompting, task-specific instructions, or human inputs, can be employed for task decomposition.'